# Binary Classification Model for Disease Prediction


# Part 1: Installation of necessary libraries

In [1]:
import numpy as np
import pandas as pd
import os 
import random
import tensorflow
import pyplot as py

# Part 2: Download of patient data with corresponding disease status

# Part 3 : Exploratory Data Analysis


# Part 4 : Generating New Data Features

# Part 5 : Creation of a machine learning model

# Part 6 : Analysis of models efficiency and loss values

# Part 7 : Log of models